In [ ]:
import time
import pyaudio

import numpy as np

In [ ]:
# import wave
# import contextlib

# from scipy.io import wavfile
# from pydub import AudioSegment

import os
import sys
import ctypes

import numpy as np
from ctypes import util

lib_path = util.find_library("rnnoise")
if (not("/" in lib_path)):
    lib_path = (os.popen('ldconfig -p | grep '+lib_path).read().split('\n')[0].strip().split(" ")[-1] or ("/usr/local/lib/"+lib_path))

lib = ctypes.cdll.LoadLibrary(lib_path)
lib.rnnoise_process_frame.argtypes = [ctypes.c_void_p,ctypes.POINTER(ctypes.c_float),ctypes.POINTER(ctypes.c_float)]
lib.rnnoise_process_frame.restype = ctypes.c_float
lib.rnnoise_create.restype = ctypes.c_void_p
lib.rnnoise_destroy.argtypes = [ctypes.c_void_p]

# borrowed from here 
# https://github.com/Shb742/rnnoise_python
class RNNoise(object):
    def __init__(self):
        self.obj = lib.rnnoise_create(None)
    def process_frame(self,inbuf):
        outbuf = np.ndarray((480,), 'h', inbuf).astype(ctypes.c_float)
        outbuf_ptr = outbuf.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
        VodProb =  lib.rnnoise_process_frame(self.obj,outbuf_ptr,outbuf_ptr)
        return (VodProb,outbuf.astype(ctypes.c_short).tobytes())

    def destroy(self):
        lib.rnnoise_destroy(self.obj)

denoiser = RNNoise()

In [ ]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000
CHUNK = 960/2 # CHUNK = 1024

audio = pyaudio.PyAudio()
denoiser = RNNoise()

In [ ]:
def get_stream_index():

    output = os.popen("pacmd list-sink-inputs")

    index = 0
    index_pid = 0
    py_pid = int(os.getpid())

    for line in output.readlines():
        line = line.strip()
        if line.startswith("index:"):
            index = int(line[7:])

        s, e = 0, 0
        if line.startswith("application.process.id"):
            for i, c in enumerate(line):
                if c == "\"":
                    if not s:
                        s = i
                    else:
                        e = i
                        break
            assert s and e
            index_pid = int(line[s + 1:e])

        if index_pid == py_pid:
            return(index)
        
    raise RuntimeError

In [ ]:
def get_sink_id():
    
    output = os.popen("pactl list short sinks")
    
    for line in output.readlines():
        info = line.split("\t")
        if info[1] == "virtsink":
            return int(info[0])
    
    raise RuntimeError

In [ ]:
def callback(in_data, frame_count, time_info, status):
    denoised_frame = denoiser.process_frame(in_data)[1]
    # print('a')
    return (denoised_frame, pyaudio.paContinue)

stream = audio.open(format=FORMAT,\
                    channels=CHANNELS,\
                    rate=RATE,\
                    input=True,\
                    output=True,\
                    stream_callback=callback,\
                    # input_device_index=1, \
                    frames_per_buffer=CHUNK)

sink_id = get_sink_id()

stream.start_stream()

stream_index = get_stream_index()
os.popen("pactl move-sink-input {} {}".format(stream_index, sink_id))

time.sleep(120)

stream.stop_stream()
stream.close()